In [7]:
import tensorflow as tf
import numpy as np
import math
import matplotlib.pyplot as plt


def batch_generator(x, y, batch_size):
    X_all = x
    y_all = y
    data_len = len(X_all)
    idx = np.arange(data_len)
    np.random.shuffle(idx)
    X_shuf = np.zeros(X_all.shape)
    Y_shuf = np.zeros(y_all.shape)
    for i in range(data_len):
        X_shuf[i] = X_all[idx[i]]
        Y_shuf[i] = y_all[idx[i]]
    X_all_padded = np.concatenate([X_shuf, X_shuf[:batch_size]], axis=0)
    y_all_padded = np.concatenate([Y_shuf, Y_shuf[:batch_size]], axis=0)
    x_batched = []
    y_batched = []
    for slice_i in range(int(math.ceil(data_len / batch_size))):
        idx = slice_i * batch_size
        X_batch = X_all_padded[idx:idx + batch_size]
        y_batch = y_all_padded[idx:idx + batch_size]
        y_batch = y_batch.astype(np.float32)
        x_batched.append(X_batch)
        y_batched.append(y_batch)
    return (x_batched, y_batched)


def conv(x_):
    conv1 = tf.layers.conv2d(
        inputs=x_,
        filters=32,  # number of filters
        kernel_size=[9, 9],
        padding="same",
        activation=tf.nn.relu)

    pool1 = tf.layers.max_pooling2d(inputs=conv1,
                                    pool_size=[2, 2],
                                    strides=2)  # convolution stride

    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,  # number of filters
        kernel_size=[7, 7],
        padding="same",
        activation=tf.nn.relu)

#     pool2 = tf.layers.max_pooling2d(inputs=conv2,
#                                     pool_size=[2, 2],
#                                     strides=2)  # convolution stride
    conv3 = tf.layers.conv2d(
        inputs=conv2,
        filters=128,  # number of filters
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    pool3 = tf.layers.max_pooling2d(inputs=conv3,
                                    pool_size=[2, 2],
                                    strides=2)  # convolution stride
    conv4 = tf.layers.conv2d(
        inputs=pool3,
        filters=256,  # number of filters
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

#     pool4 = tf.layers.max_pooling2d(inputs=conv4,
#                                     pool_size=[2, 2],
#                                     strides=2)  # convolution stride
    conv5 = tf.layers.conv2d(
            inputs=conv4,
            filters=256, # number of filters
            kernel_size=[3, 3],
            padding="same",
            activation=tf.nn.relu)

    pool5 = tf.layers.max_pooling2d(inputs=conv5,
                                    pool_size=[2, 2],
                                    strides=2)  # convolution stride

    conv6 = tf.layers.conv2d(
            inputs=pool5,
            filters=512, # number of filters
            kernel_size=[3, 3],
            padding="same",
            activation=tf.nn.relu)

#     pool6 = tf.layers.max_pooling2d(inputs=conv6,
#                                     pool_size=[2, 2],
#                                     strides=2)  # convolution stride
#    conv7 = tf.layers.conv2d(
#            inputs=conv6,
#            filters=512, # number of filters
#            kernel_size=[5, 5],
#            padding="same",
#            activation=tf.nn.relu)

 #   pool7 = tf.layers.max_pooling2d(inputs=conv7,
 #                                   pool_size=[2, 2],
 #                                   strides=2)  # convolution stride
    

    pool_flat = tf.contrib.layers.flatten(pool6, scope='pool2flat')
    dense = tf.layers.dense(inputs=pool_flat, units=500, activation=tf.nn.relu)
    logits = tf.layers.dense(inputs=dense, units=3755)
    return logits


def apply_classification_loss(model_function):
    with tf.Graph().as_default() as g:
        with tf.device("/gpu:0"):  # use gpu:0 if on GPU
            x_ = tf.placeholder(tf.float32, [None, 58, 58, 1])
            y_ = tf.placeholder(tf.int32, [None])
            drop_out = tf.placeholder(tf.float32)
            y_logits = model_function(x_)

            y_dict = dict(labels=y_, logits=y_logits)
            losses = tf.nn.sparse_softmax_cross_entropy_with_logits(**y_dict)
            cross_entropy_loss = tf.reduce_mean(losses)
            trainer = tf.train.AdamOptimizer(learning_rate=0.0001)
            train_op = trainer.minimize(cross_entropy_loss)

            y_pred = tf.argmax(tf.nn.softmax(y_logits), axis=1)
            correct_prediction = tf.equal(tf.cast(y_pred, tf.int32), y_)
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    model_dict = {'graph': g, 'inputs': [x_, y_, drop_out], 'train_op': train_op,
                  'accuracy': accuracy, 'loss': cross_entropy_loss}

    return model_dict


def train_model(model_dict, epoch_n, print_every, xtr, ytr, xte, yte, drop_out, load_model=False):
    with model_dict['graph'].as_default(), tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        saver = tf.train.Saver()
        if load_model == True:
            saver.restore(sess, 'checkpoints3755/checkpoint.ckpt')
        try:
            tr_acc = []
            tr_ls = []
            te_acc = []
            te_ls = []
            for epoch_i in range(epoch_n):
                Xtr_, ytr_ = batch_generator(xtr, ytr, 256)
                for i in range(len(Xtr_)):
                    x_train_batch = Xtr_[i]
                    y_train_batch = ytr_[i]
                    train_feed_dict = dict(zip(model_dict['inputs'][:2], [x_train_batch, y_train_batch]))
                    train_feed_dict[model_dict['inputs'][2]] = drop_out
                    sess.run(model_dict['train_op'], feed_dict=train_feed_dict)
                    train_compute = [model_dict['loss'], model_dict['accuracy']]
                    train_result = sess.run(train_compute, train_feed_dict)
                    if i % print_every == 0:
                        print('Current epoch number: {}'.format(epoch_i))
                        tr_ls.append(train_result[0])
                        print('loss is {}'.format(train_result[0]))
                        tr_acc.append(train_result[1])
                        print('train_accuracy is {}'.format(train_result[1]))
                        test_acc = []
                        test_loss = []
                        for j in range(len(xte)):
                            x_test_batch = xte[j]
                            y_test_batch = yte[j]
                            test_feed_dict = dict(zip(model_dict['inputs'], [x_test_batch, y_test_batch]))
                            to_compute = [model_dict['loss'], model_dict['accuracy']]
                            result = sess.run(to_compute, test_feed_dict)
                            test_loss.append(result[0])
                            test_acc.append(result[1])
                        avg_test_loss = np.mean(test_loss)
                        avg_test_acc = np.mean(test_acc)
                        te_acc.append(avg_test_acc)
                        te_ls.append(avg_test_loss)
                        print('test loss is {}'.format(avg_test_loss))
                        print('test_accuracy is {}'.format(avg_test_acc))
                        print()
        except KeyboardInterrupt:
            saver.save(sess, 'checkpoints3755/checkpoint.ckpt')
            return (tr_ls, tr_acc, te_ls, te_acc)
        # train_loss.append(train_result[0])
        # train_accuracy.append(train_result[1])
        # test_loss.append(result[0])
        # test_accuracy.append(result[1])
        # loss.append(result[0])
        # accuracy.append(result[1])
        # except KeyboardInterrupt:
        #     saver.save(sess, 'checkpoints3755/checkpoint.ckpt')

In [ ]:
x_te = np.load('test_image.npy')
y_te = np.load('test_label.npy')
x_tr = np.load('train_image.npy')
y_tr = np.load('train_label.npy')
x_te = np.reshape(x_te, [-1, 58, 58, 1])
x_tr = np.reshape(x_tr, [-1, 58, 58, 1])

# Xtr_, ytr_ = batch_generator(x_tr, y_tr, 256)
Xte_, yte_ = batch_generator(x_te, y_te, 256)

model_dict = apply_classification_loss(conv)

tr_ls, tr_acc, te_ls, te_acc = train_model(model_dict, epoch_n=1000000, print_every=350, xtr=x_tr, ytr=y_tr, xte=Xte_, yte=yte_, drop_out=0.1)

tr_ls.remove(tr_ls[-1])
tr_acc.remove(tr_acc[-1])

batch_num = []
for m in range(len(tr_ls)):
    batch_num.append(m)

plt.subplot(2, 1, 1)
plt.plot(batch_num, tr_ls, color='r')
plt.plot(batch_num, te_ls, color='k')
plt.title('train test loss and accuracy against batch number')
plt.ylabel('loss')

plt.subplot(2, 1, 2)
plt.plot(batch_num, tr_acc, color='r')
plt.plot(batch_num, te_acc, color='k')
plt.xlabel('batch_num')
plt.ylabel('accuracy')
plt.show()

print('done')

Current epoch number: 0
loss is 8.05411434173584
train_accuracy is 0.00390625
test loss is 8.339137077331543
test_accuracy is 0.00025892857229337096

Current epoch number: 0
loss is 8.228825569152832
train_accuracy is 0.0
test loss is 8.231148719787598
test_accuracy is 0.00033928570337593555

Current epoch number: 0
loss is 8.229683876037598
train_accuracy is 0.0
test loss is 8.230813980102539
test_accuracy is 0.0003124999930150807

Current epoch number: 0
loss is 8.222567558288574
train_accuracy is 0.0
test loss is 8.228887557983398
test_accuracy is 0.0003348214377183467

Current epoch number: 0
loss is 8.10012435913086
train_accuracy is 0.0
test loss is 8.103140830993652
test_accuracy is 0.0007008928805589676

Current epoch number: 0
loss is 5.183834075927734
train_accuracy is 0.0859375
test loss is 5.51600456237793
test_accuracy is 0.05342857167124748

Current epoch number: 0
loss is 2.6963253021240234
train_accuracy is 0.375
test loss is 2.987971067428589
test_accuracy is 0.3223839

test loss is 0.4102028012275696
test_accuracy is 0.9019285440444946

Current epoch number: 5
loss is 0.013438060879707336
train_accuracy is 1.0
test loss is 0.46688002347946167
test_accuracy is 0.9034821391105652

Current epoch number: 5
loss is 0.022900842130184174
train_accuracy is 0.99609375
test loss is 0.44681376218795776
test_accuracy is 0.8994598388671875

Current epoch number: 5
loss is 0.022427858784794807
train_accuracy is 0.9921875
test loss is 0.4567965269088745
test_accuracy is 0.9002231955528259

Current epoch number: 5
loss is 0.017285387963056564
train_accuracy is 1.0
test loss is 0.4782869815826416
test_accuracy is 0.9007589221000671

Current epoch number: 5
loss is 0.03787659853696823
train_accuracy is 0.9921875
test loss is 0.4272017776966095
test_accuracy is 0.9014151692390442

Current epoch number: 5
loss is 0.03491884097456932
train_accuracy is 0.99609375
test loss is 0.44773343205451965
test_accuracy is 0.8991339206695557

Current epoch number: 5
loss is 0.033258